In [1]:
!az login

[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "1fdb6d07-480b-4c31-a913-e847d51e7446",
    "id": "ea54fca2-3a3e-4b3b-91e7-a7bf971e0443",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantId": "1fdb6d07-480b-4c31-a913-e847d51e7446",
    "user": {
      "name": "Philliplakaschus@gmail.com",
      "type": "user"
    }
  }
]


In [6]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.consumption import ConsumptionManagementClient

import os
from dotenv import load_dotenv
load_dotenv()

credential = DefaultAzureCredential()
subscription_id = os.environ["AZURE_SUBSCRIPTION_ID"]

In [7]:
resource_client = ResourceManagementClient(credential, subscription_id)
consumption_client = ConsumptionManagementClient(credential, subscription_id)

# Fetch the list of resource groups in the subscription
resource_groups = resource_client.resource_groups.list()

# Iterate through the resource groups and fetch the resources in each group
for resource_group in resource_groups:
    resources = resource_client.resources.list_by_resource_group(resource_group.name)
    for resource in resources:
        print(f"Resource: {resource.name}, Type: {resource.type}")


Resource: cs110032002dbe11b2c, Type: Microsoft.Storage/storageAccounts
Resource: DefaultWorkspace-koreacentral, Type: Microsoft.OperationalInsights/workspaces
Resource: ai102-test, Type: Microsoft.CognitiveServices/accounts
Resource: ai102-keys, Type: Microsoft.KeyVault/vaults
Resource: DefaultWorkspace-eastus, Type: Microsoft.OperationalInsights/workspaces
Resource: DefaultWorkspace-eastus2, Type: Microsoft.OperationalInsights/workspaces
Resource: azureapp-auto, Type: Microsoft.Insights/actiongroups
Resource: Service Health-Problem in „Azure subscription 1“, Type: Microsoft.Insights/activityLogAlerts


In [26]:
# Get the current costs for the subscription
scope = f"/subscriptions/{subscription_id}"
query = "usageStart ge 2024-01-01 and properties/usageEnd le 2024-12-31"
usage_details = consumption_client.usage_details.list(scope, filter=query)

total_cost = 0
for usage in usage_details:
    total_cost += usage.cost_in_billing_currency

print(f"Current total cost: ${total_cost:.2f}")


Current total cost: $0.00


In [ ]:
# Get the cost forecast for the subscription
forecast_scope = f"/subscriptions/{subscription_id}/providers/Microsoft.Consumption/forecasts"
forecast_period = "Monthly"
forecast_include_actual_cost = True
forecast_include_fresh_partial_cost = True

forecast = consumption_client.forecasts.usage(
    scope=forecast_scope,
    grain=forecast_period,
    include_actual_cost=forecast_include_actual_cost,
    include_fresh_partial_cost=forecast_include_fresh_partial_cost
)

print(f"Forecast cost for the next month: ${forecast.amount:.2f}")